# TFDV
tensorflow data validation訳してTFDVのやれることを確認していく

## Computing descriptive data statistics
基本的な統計量をすぐ測れる

In [1]:
import tensorflow_data_validation as tfdv

/Users/baito/Works/TFX/.venv/lib/python3.7/site-packages/apache_beam/__init__.py:84: UserWarning: Some syntactic constructs of Python 3 are not yet fully supported by Apache Beam.
  'Some syntactic constructs of Python 3 are not yet fully supported by '
/Users/baito/Works/TFX/.venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/baito/Works/TFX/.venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/baito/Works/TFX/.venv/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing 

In [12]:
stats = tfdv.generate_statistics_from_tfrecord('./tfrecord/train_transformed-00000-of-00001')

In [13]:
tfdv.visualize_statistics(stats)

ｃｓｖから読み込む事もできる。（この場合一行目がカラム名でないのでexplicitに入力する必要がある）

In [10]:
stats2 = tfdv.generate_statistics_from_csv('./data/taxi-train.csv', column_names=['dropofflon','dropofflat','passengers', 'fare_amount', 'pickuplon','pickuplat', 'key'])

In [11]:
tfdv.visualize_statistics(stats2)

## Inferring a schema over the data
> 特徴量がどういった属性を持っているかを述べたスキーマを作成出来る。プロパティとして
* あると期待されている特徴量かどうか
* 型
* それぞれのイグザンプルの中の1つの特徴量の値の数
* 全てのイグザンプルをまたいだそれぞれの特徴量があるかどうか
* 特徴量が期待されたドメインかどうか　

ようは簡単に言えば正しいデータとはどういうものかを記述する。これらはエラーを発見するのに使われる。またこれらのスキーマはTFTでも使われる。
   

In [14]:
# schemaを書く
schema = tfdv.infer_schema(satas)

In [16]:
schema

feature {
  name: "dropofflat"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "dropofflon"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "passengers"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "fare_amount"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "pickuplon"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "pickuplat"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "key"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1

In [17]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'dropofflat',INT,required,,-
'dropofflon',INT,required,,-
'passengers',FLOAT,required,,-
'fare_amount',FLOAT,required,,-
'pickuplon',INT,required,,-
'pickuplat',INT,required,,-
'key',FLOAT,required,,-


In [21]:
tfdv.get_feature(schema, 'dropofflat').presence.min_fraction = 0.5

In [23]:
schema

feature {
  name: "dropofflat"
  type: INT
  presence {
    min_fraction: 0.5
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "dropofflon"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "passengers"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "fare_amount"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "pickuplon"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "pickuplat"
  type: INT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1
    }
  }
}
feature {
  name: "key"
  type: FLOAT
  presence {
    min_fraction: 1.0
    min_count: 1
  }
  shape {
    dim {
      size: 1

`dropofflat`の`presense`の値が変わっていることがわかる

## Checking the data for errors 
データセットが期待したものであるか、どこが通常と違うのかを測定できる。

### Matching the statistics of the dataset against a schema
異なるデータに対して統計量があっているかをスキーマと比較して検出する

In [24]:
other_stats = tfdv.generate_statistics_from_tfrecord('./tfrecord/test_transformed-00000-of-00001')

In [26]:
anomalies = tfdv.validate_statistics(statistics=other_stats, schema=schema)
tfdv.display_anomalies(anomalies)

特におかしい特徴量は無いようだ。試しに確実にエラーがでるCSVでやってみる。

In [27]:
anomalies2 = tfdv.validate_statistics(statistics=stats2, schema=schema)
tfdv.display_anomalies(anomalies2)

,Anomaly short description,Anomaly long description
Feature name,,
'pickuplon',Expected data of type: INT but got FLOAT,
'dropofflat',Expected data of type: INT but got FLOAT,
'dropofflon',Expected data of type: INT but got FLOAT,
'key',Expected data of type: FLOAT but got INT,


## Checking for erros on a per-example basis
TFDVは一つ一つのエグザンプルに対して異常値検出をすることも可能である。

In [28]:
options = tfdv.StatsOptions(schema=schema)
anomalous_example_stats = tfdv.validate_tfexamples_in(
   data_location='./tfrecord/test_transformed-00000-of-00001', stats_options=options) # python 3ではまだ未対応
tfdv.display_anomalies(anomalous_example_stats)

AttributeError: module 'tensorflow_data_validation' has no attribute 'validate_tfexamples_in_tfrecord'

## Checking data skew and drift

In [33]:
serving_stats = tfdv.generate_statistics_from_tfrecord('./tfrecord/test_transformed-00000-of-00001')

tfdv.get_feature(schema, 'passengers').skew_comparator.infinity_norm.threshold = 0.01
skew_anomalies = tfdv.validate_statistics(
    statistics=stats, schema=schema, serving_statistics=serving_stats)

In [34]:
tfdv.display_anomalies(skew_anomalies)